<a href="https://colab.research.google.com/github/jorgeportilla202231013/Test/blob/MCS-2022/ML_Proyecto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
import pandas as pd
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['forestfires.csv']))
print(df)

Saving forestfires.csv to forestfires (1).csv
     X  Y month  day  FFMC    DMC     DC   ISI  temp  RH  wind  rain   area
0    7  5   mar  fri  86.2   26.2   94.3   5.1   8.2  51   6.7   0.0   0.00
1    7  4   oct  tue  90.6   35.4  669.1   6.7  18.0  33   0.9   0.0   0.00
2    7  4   oct  sat  90.6   43.7  686.9   6.7  14.6  33   1.3   0.0   0.00
3    8  6   mar  fri  91.7   33.3   77.5   9.0   8.3  97   4.0   0.2   0.00
4    8  6   mar  sun  89.3   51.3  102.2   9.6  11.4  99   1.8   0.0   0.00
..  .. ..   ...  ...   ...    ...    ...   ...   ...  ..   ...   ...    ...
512  4  3   aug  sun  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0   6.44
513  2  4   aug  sun  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0  54.29
514  7  4   aug  sun  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0  11.16
515  1  4   aug  sat  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0   0.00
516  6  3   nov  tue  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0   0.00

[517 rows x 13 columns]


In [201]:
RANDOM_STATE = 42
TEST_SIZE = 0.2

In [202]:
numeric_columns = ["FFMC", "DMC", "DC", "ISI", "temp", "RH", "wind", "rain"]

In [204]:
class LinearRegression:
    def __init__(self, epochs=100, alpha=0.01, reg = "L1", l = 0.1, p = 0.5 , error_function="MSE"):
        self.epochs = epochs
        self.alpha = alpha
        self.error_function = error_function
        self.reg = reg
        self.l = l
        self.p = p
    
    def h(self, X, W):
        return X @ W

    def error(self, X, W, y, l):
        n = X.shape[0]
        p = self.p
        if self.reg == "False":
            return np.sum((y - self.h(X, W)) ** 2) / (2 * n)
        elif self.reg == "L2":
            return np.sum((y - self.h(X, W)) ** 2) / (2 * n) + l*np.sum(W ** 2)
        elif self.reg == "L1":
            return np.sum((y - self.h(X, W)) ** 2) / (2 * n) + l*np.sum(W)
        elif self.reg == "Elastinet":
            return np.sum((y - self.h(X, W)) ** 2) / (2 * n) + p*l*np.sum(W ** 2) + (1-p)*l*np.sum(W)

    def derivada(self, X, W, y, l):
        n = y.size
        p = self.p

        if self.reg == "False":
            return (1/n) * X.T.dot(self.h(X, W) - y)
        elif self.reg == "L2":
            return (1/n) * X.T.dot(self.h(X, W) - y) + 2*l*W
        elif self.reg == "L1":
            return (1/n) * X.T.dot(self.h(X, W) - y) + l
        elif self.reg == "Elastinet":
            return (1/n) * X.T.dot(self.h(X, W) - y) + 2*l*W*p + (1-p)*l
    
    def update(self, W, dW, alpha):
        return W - alpha * dW

    def fit(self, X, y):
        X, y = self._fix_dimensions(X, y)
        W = np.random.rand(X.shape[1], 1)
        L = self.error(X, W, y, self.l)
        losses = []
        i = 0
        while i < self.epochs:
            dW = self.derivada(X, W, y, self.l)
            W = self.update(W, dW, self.alpha)
            L = self.error(X, W, y, self.l)
            losses.append(L)
            i += 1
        
        self.W = W
        self.losses = losses
        
        return W, losses

    def predict(self, X):
        return self.h(self._fix_dimensions(X)[0], self.W)
    
    def _fix_dimensions(self, X, y=None):
        # Agregamos columna de unos a X
        ones = np.ones(X.shape[0])
        X = np.c_[ones, X]
        if y is not None:
            y = y.reshape(y.shape[0], -1)
        return X, y

In [205]:
# Funcion usada para encodear variables categoricas(month, day, etc)
def encode(X, encoding_method):
    df = pd.DataFrame()
    for c in X.columns:
        if X[c].dtype == 'object':
            if encoding_method == "label_encoding":
                le = LabelEncoder()
                df[c] = le.fit_transform(X[c])
            elif encoding_method == "ordinal_encoding":
                oe = OrdinalEncoder()
                df[c] = np.squeeze(oe.fit_transform(X[[c]].values))
            else:
                df[c] = X[c]
        else:
            df[c] = X[c]

    if encoding_method == "one_hot_encoding":
        df = pd.get_dummies(df)
    return df

# Funcion usada para normalizar variables numericas
def scale(df, scaling_method):
    scaler = None
    if scaling_method == "standard":
        scaler = StandardScaler()
    elif scaling_method == "minmax":
        scaler = MinMaxScaler()
    
    numeric_columns = df.select_dtypes(exclude=["object"]).columns
    if scaler:
        df[numeric_columns] = scaler.fit_transform(df[numeric_columns].values)
    
    return df

# Funcion usada para detectar outliers. Usamos DBSCAN para generar los clusters
# y las observaciones que no pertenecen a ningun cluster(label = -1) se consideran outliers
def detect_outliers(df):
    outlier_detector = DBSCAN(eps=4, metric='euclidean', min_samples=2)
    clusters = outlier_detector.fit(df.values.reshape(-1,1))
    labels = clusters.labels_
    return np.where(labels == -1)[0]

def remove_outliers(df):
    outliers = []
    for column in numeric_columns:
        outliers.extend(list(detect_outliers(df[column])))
    outliers = list(set(outliers))
    df.drop(labels=outliers, axis=0, inplace=True)
    return df

def preprocessing(df, encoding_method, scaling_method):
    df = scale(df, scaling_method)
    df = encode(df, encoding_method)
    return df


"""
Corremos el modelo usando una configuracion determinada.

X: observaciones
y: target variable
params: parametros a pasar al modelo de regresion
encoding: metodo a utilizar para encodear las variables categoricas
scaling: metodo a utilizar para normalizar las variables numericas
cols_to_drop: columnas a eliminar del dataset
remove_outliers: flag que indica si debemos eliminar los outliers o no

"""
def run_model(df, d, params, encoding="ordinal_encoding", scaling="standard", cols_to_drop=[], drop_outliers=False):
    if drop_outliers:
        df = remove_outliers(df)
    
    X = df.drop(["area"], axis=1)
    y = df["area"].values
    
    X = X.drop(cols_to_drop, axis=1)

    Xp2 = encode(X, encoding)

    poly = PolynomialFeatures(degree = d)
    Xp3 = poly.fit_transform(Xp2)

    Xp4 = pd.DataFrame(Xp3)

    Xp = scale(Xp4, scaling)

    X_train, X_test, y_train, y_test = train_test_split(Xp, y, random_state=RANDOM_STATE, test_size=TEST_SIZE)
        
    model = LinearRegression(**params)
    w, loss = model.fit(X_train, y_train)
    preds = model.predict(X_test)
    n = len(y_test)
    y_test = y_test.reshape(y_test.shape[0],1)
    if params["reg"] == "False":
          return (1/(2*n))*np.sum((y_test - preds) ** 2)
    elif params["reg"] == "L2":
          return (1/(2*n))*np.sum((y_test - preds) ** 2) + params["l"]*np.sum(w ** 2)
    elif params["reg"] == "L1":
          return (1/(2*n))*np.sum((y_test - preds) ** 2) + params["l"]*np.sum(np.abs(w))
    elif params["reg"] == "Elastinet":
          return (1/(2*n))*np.sum((y_test - preds) ** 2) + 0.5*params["l"]*np.sum(w ** 2) + 0.5*params["l"]*np.sum(np.abs(w))

    mse = (1/(2*n))*np.sum((y_test.reshape(y_test.shape[0],1) - preds) ** 2)

    return mse


In [267]:
def run_experiments(drop_outliers=True, d = 2):
    epochs_list = [100]
    error_functions = ["MSE"]
    alphas = [0.01]
    lamb = [0.1,1,10]
    reg_list = ["False", "L1", "L2", "Elastinet"]
    results = []
    for epochs in epochs_list:
      for reg in reg_list:
        for alpha in alphas:
          for l in lamb:
            for error_function in error_functions:
                print(
                    "alpha:", alpha,
                    "lambda:", l,
                    "reg:", reg,
                    ", epochs:", epochs,
                    ", remove_outliers:", drop_outliers,
                    ", error_function:", error_function
                )
                params = {"alpha": alpha, "epochs": epochs, "error_function": error_function, "l": l, "reg": reg}
                mse = run_model(df.copy(), d, params=params, drop_outliers=drop_outliers)
                print()
                results.append([epochs, alpha, l, reg, mse])
    return results, y_test, preds

In [262]:
results, y_test, preds = run_experiments(drop_outliers=True, d = 2)
results

alpha: 0.01 lambda: 0.1 reg: False , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 1 reg: False , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 10 reg: False , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 0.1 reg: L1 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 1 reg: L1 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 10 reg: L1 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 0.1 reg: L2 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 1 reg: L2 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 10 reg: L2 , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 0.1 reg: Elastinet , epochs: 100 , remove_outliers: True , error_function: MSE

alpha: 0.01 lambda: 1 reg: Elastinet , epochs: 100 , remove_out

[[100, 0.01, 0.1, 'False', 364.60307752530775],
 [100, 0.01, 1, 'False', 362.8569667420488],
 [100, 0.01, 10, 'False', 364.8214767108059],
 [100, 0.01, 0.1, 'L1', 369.03044214467064],
 [100, 0.01, 1, 'L1', 424.29415423213055],
 [100, 0.01, 10, 'L1', 2275.5744245426417],
 [100, 0.01, 0.1, 'L2', 372.5584687109545],
 [100, 0.01, 1, 'L2', 401.9009867620491],
 [100, 0.01, 10, 'L2', 391.7073876864267],
 [100, 0.01, 0.1, 'Elastinet', 371.6333854526737],
 [100, 0.01, 1, 'Elastinet', 413.02880775507566],
 [100, 0.01, 10, 'Elastinet', 516.7110640255612]]